# Transform Phenotypic Data into a Common Format
This script accesses the extracted data in their original format and applys a mapping to harmonize the data into a common tabular format.

In [9]:
import os
import json

import numpy as np
import pandas as pd

print(os.path.abspath(os.path.curdir))

/Users/nicholsn/Repos/metasearch/crawler/transform


In [10]:
# Path to projects with extracted files.
extract_path = os.path.abspath('../extract')
# Path to where the tranformed projects will go.
xfm_path = os.path.abspath(os.path.curdir)
# Project dirs.
project_names = [i for i in os.listdir(extract_path) if '.' not in i]
extract_dirs = [os.path.join(extract_path, i) for i in project_names]
xfm_dirs = [os.path.join(xfm_path, i) for i in project_names]

In [20]:
def apply_mapping(mapping, df_list):
    """
    Reads a dictionary mapping and list of dataframe, then merges the
    dataframes and convers the column name and values into a common
    format. Note: the dataframes should have a common structure.
    
    Example Mapping Structure
    =========================
    {  
        "DX_GROUP":
      {
        "element": "diagnosis",
        "type": "category",
        "1": "autism",
        "2": "control"
      }
    }
    """
    results = list()
    csv = pd.concat(df_list)
    csv.reset_index(drop=True, inplace=True)
    for col, elem in mapping.iteritems():
        # Use get in case the col is missing/static.
        series = csv.get(col)
        # Categories are mapped to common values.
        if elem.get('type') == 'category':
            result = series.apply(lambda x: elem.get(str(x)))
        # Values parsed as numbers are checked for any mappings (e.g., -999 == NaN).
        elif elem.get('type') == 'number':
            result = series.apply(lambda x: np.NaN if str(x) in elem.keys() else x)
        # Used to create a column of all the same value.
        elif elem.get('type') == 'static':
            val = [elem.get('value')] * csv.shape[0]
            result = pd.Series(val, index=csv.index)
        else:
            try:
                # Handle IDs being read as float.
                result = series.apply(lambda x: str(int(x)) if pd.notnull(x) else x)
            except ValueError as e:
                result = series
        # Concat all csv in a extract dir into one dataframe.        
        df = pd.DataFrame()
        df[elem.get('element')] = result
        results.append(df)
        concat = pd.concat(results, axis=1)
    # Merge any columns with duplicate names to fil in NaN from concat step.
    return concat.groupby(level=0, axis=1).first()

In [21]:
# Read a mapping.json file from each sub directory to process.
for project_name in project_names:
    df_list = list()
    mapping = dict()
    extract_dir = os.path.join(extract_path, project_name)
    xfm_dir = os.path.join(xfm_path, project_name)
    # All files must use the same data dictionary in a given directory.
    extract_files = [os.path.join(extract_dir, i) for i in os.listdir(extract_dir)]
    mapping_file = os.path.join(xfm_dir, 'mapping.json')
    if os.path.exists(mapping_file):
        with open(mapping_file, 'rb') as fi:
            mapping.update(json.load(fi))
        ext_type = dict(csv=',', tsv='\t')
        for extract_file in extract_files:
            # Process each file.
            ext = extract_file.split('.')[-1]
            sep_type = ext_type.get(ext)
            df = pd.read_csv(extract_file, sep=sep_type)
            df_list.append(df)
        xfm = apply_mapping(mapping, df_list)
        pheno_file = ''.join([project_name, '_', 'phenotype.csv'])
        pheno_path = os.path.join(xfm_dir, pheno_file)
        # Add a column for the specific project.
        xfm['project'] = project_name
        # If csv file didn't have a site column then use the project name.
        if 'site_id' not in xfm.columns:
            xfm['site_id'] = pd.np.NaN
        xfm.to_csv(pheno_path, index=False)